### 실습중인 쇼핑몰 상품은 여러페이지에 걸쳐 진열되어 있음
    - 함수를 구성 후 여러페이지 크롤링
    - 에러나는 부분은 함수를 수정해가면서 진행
    - 함수를 기능별로 세분화

In [1]:
import pandas as pd
from bs4 import BeautifulSoup 
import requests
from urllib.request import urlopen
import numpy as np
import ssl

In [2]:
# 사이트 url
url = 'http://jolse.com/category/toners-mists/1019/'

### 함수를 기능별로 세분화
- 요청 및 파싱 객체 생성 : get_request_product(url), 반환값-bs4 객체
- 전체 정보 추출 : get_page_product(bs_obj)
    - bs_obj를 전달받아서
    - get_proudct_info(box)를 호출(제품 1개에 대한 정보가 반환)
    - 1페이지 제품에 대한 df을 생성 후 반환
- 각 제품의 정보 추출 : get_proudct_info(box)
    - 제품정보 1개에 대해 제품명/가격/세일가격을 추출해서 반환

In [3]:
# df 생성
prd_dict = {'품목':[], '가격':[], '세일가격':[]}
prd_df = pd.DataFrame(prd_dict)
prd_df

,품목,가격,세일가격


In [4]:
# 1. 요청 및 파싱 객체(bs4) 생성 함수
def get_request_product(url):
    # 요청 후 코드 추출
    url = url
    context = ssl._create_unverified_context()
    htmls = urlopen(url, context=context)
    htmls = htmls.read()
    # bs 객체 생성
    bs_obj = BeautifulSoup(htmls, 'html.parser')
    return bs_obj

In [5]:
# 전체 정보 추출 : get_page_product(bs_obj)
# bs_obj를 전달받아서
# get_proudct_info(box)를 호출(제품 1개에 대한 정보가 반환)
# 전역변수 df 활용해서 제품 정보 저장
# 최초 호출 함수
def get_page_product(url):
    global prd_df 
    # global 변수를 함수 내부가 아닌 외부에서 찾을 것이라는 의미
    # global은 전역변수(함수내/외부에서 모두 사용)로 생성
    bs_obj = get_request_product(url)
    # 페이지 내에 전체 제품 정보 추출
    boxes = bs_obj.findAll('div',{'class': 'description'})
    # 각 제품에 대한 정보 추출 후 df에 저장
    for box in boxes[2:]:  # best 제외 => best는 다음페이지를 가도 나타나기 때문에 제외하고 추출
        res = pd.DataFrame(get_product_info(box), index=range(1,2))
        prd_df = pd.concat([prd_df,res], axis=0, ignore_index=True)

In [6]:
# 각 제품 정보 추출하는 함수
# box 는 class 값이 description인 div 태그 1개가 전달
def get_product_info(box):
    try:
        title = box.find('strong', {'class':'name'}).text.split(':')[1].strip()
        lis = box.find('ul').findAll('li')
        price = (lis[0].text.split(' ')[2])
        sale_price = (lis[1].text.split(' ')[2])    
        return {'품목':title, '가격':price, '세일가격':sale_price}
    except:
        print('error')

In [7]:
url

'http://jolse.com/category/toners-mists/1019/'

In [8]:
# 함수 호출 테스트 
get_page_product(url)

In [9]:
prd_df

,품목,가격,세일가격
0,Isntree Hyaluronic Acid Toner 200ml (Renewal),19.30,17.37
1,SOME BY MI AHA BHA PHA 30 Days Miracle Toner 1...,24.00,14.99
2,SOME BY MI Galactomyces Pure Vitamin C Glow To...,34.00,30.60
3,ROUND LAB 1025 Dokdo Toner 200ml,17.00,15.30
4,SOME BY MI Snail Truecica Miracle Repair Toner...,24.00,21.60
5,COSRX AHA 7 WHITEHEAD POWER LIQUID 100ml,17.81,16.03
6,secretKey Fresh Toner 248ml,16.00,14.40
7,BLITHE Vital Treatment 8 Nourishing Beans 150ml,42.90,38.61
8,COSRX Two In One Poreless Power Liquid 100ml,22.00,19.80
9,Elizavecca Hell-Pore Clean up AHA Fruit Toner ...,14.00,12.60


## 여러 페이지 추출
- 해당 쇼핑몰의 url 분석
- 각 페이지에 대하여 url에 파라미터가 다르게 전송됨
- https://jolse.com/category/toners-mists/1019?page= + 페이지번호
- url 작성시 페이지번호를 따로 부착

In [10]:
base_url = 'http://jolse.com/category/toners-mists/1019?page='
# 2페이지부터 4페이지까지 제품을 추출
for i in range(2,5):
    url = base_url + str(i)
    get_page_product(url)

In [11]:
prd_df

,품목,가격,세일가격
0,Isntree Hyaluronic Acid Toner 200ml (Renewal),19.30,17.37
1,SOME BY MI AHA BHA PHA 30 Days Miracle Toner 1...,24.00,14.99
2,SOME BY MI Galactomyces Pure Vitamin C Glow To...,34.00,30.60
3,ROUND LAB 1025 Dokdo Toner 200ml,17.00,15.30
4,SOME BY MI Snail Truecica Miracle Repair Toner...,24.00,21.60
...,...,...,...
75,Papa Recipe Eggplant Clearing Peeling Pad Tone...,28.00,25.20
76,AXIS-Y Biome Comforting Infused Toner 200ml,27.00,24.30
77,Isntree YAM ROOT VEGAN MILK TONER 200ml,24.10,21.69
78,BENTON Aloe BHA Skin Toner 200ml (22AD),19.00,12.99


In [12]:
# 수집데이터 저장
# prd_df.to_csv('./crawl_data/beauty_price.csv')

## 연습문제 
1. best 제품을 슬라이싱이 아닌 best 제품의 진열대(ul 태그)를 제외하고 
2. 모든 제품 진열대(ul)를 이용해서 제품을 추출하도록 함수를 수정
3. 전체 페이지가 몇 페이지인지 확인하는 크롤링 코드를 작성해서 전체 페이지를 얻어오고
    - 3번에서 얻어온 전체 페이지 수 만큼 크롤링되게 수정

### 연습문제 작성 후 제출폴더에 현재 파일 제출(파일 제목 변경 후 제출)

In [13]:
import pandas as pd
from bs4 import BeautifulSoup 
import requests
from urllib.request import urlopen
import numpy as np
import ssl

In [14]:
# df 생성
prd_dict = {'품목':[], '가격':[], '세일가격':[]}
prd_df = pd.DataFrame(prd_dict)
prd_df

,품목,가격,세일가격


In [15]:
# 1. 요청 및 파싱 객체(bs4) 생성 함수
def get_request_product(url):
    # 요청 후 코드 추출
    url = url
    context = ssl._create_unverified_context()
    htmls = urlopen(url, context=context)
    htmls = htmls.read()
    # bs 객체 생성
    bs_obj = BeautifulSoup(htmls, 'html.parser')
    return bs_obj

In [16]:
# 각 제품 정보 추출하는 함수
# box 는 class 값이 description인 div 태그 1개가 전달
def get_product_info(box):
    try:
        title = box.find('strong', {'class':'name'}).text.split(':')[1].strip()
        lis = box.find('ul').findAll('li')
        price = (lis[0].text.split(' ')[2])
        sale_price = (lis[1].text.split(' ')[2])    
        return {'품목':title, '가격':price, '세일가격':sale_price}
    except:
        print('error')

In [17]:
# 호출 시작 함수
def get_page_product(url):
    global prd_df 
    # global 변수를 함수 내부가 아닌 외부에서 찾을 것이라는 의미
    # global은 전역변수(함수내/외부에서 모두 사용)로 생성
    bs_obj = get_request_product(url)
    # 페이지 내에 전체 제품 정보 추출
    page = bs_obj.findAll('ul', {'class':'prdList grid5'})[1]
    boxes = page.findAll('div',{'class': 'description'})
    # 각 제품에 대한 정보 추출 후 df에 저장
    for box in boxes: 
        res = pd.DataFrame(get_product_info(box), index=range(1,2))
        prd_df = pd.concat([prd_df,res], axis=0, ignore_index=True)

In [18]:
url = 'http://jolse.com/category/toners-mists/1019?page=1'

In [19]:
# 함수 호출 테스트
# prd_df에 전달된 url 페이지의 일반 진열 제품의 정보를 저장해주는 함수
get_page_product(url)

In [20]:
# 베스트 상품 제외하고 추출 되는 것 확인
prd_df

,품목,가격,세일가격
0,Isntree Hyaluronic Acid Toner 200ml (Renewal),19.30,17.37
1,SOME BY MI AHA BHA PHA 30 Days Miracle Toner 1...,24.00,14.99
2,SOME BY MI Galactomyces Pure Vitamin C Glow To...,34.00,30.60
3,ROUND LAB 1025 Dokdo Toner 200ml,17.00,15.30
4,SOME BY MI Snail Truecica Miracle Repair Toner...,24.00,21.60
5,COSRX AHA 7 WHITEHEAD POWER LIQUID 100ml,17.81,16.03
6,secretKey Fresh Toner 248ml,16.00,14.40
7,BLITHE Vital Treatment 8 Nourishing Beans 150ml,42.90,38.61
8,COSRX Two In One Poreless Power Liquid 100ml,22.00,19.80
9,Elizavecca Hell-Pore Clean up AHA Fruit Toner ...,14.00,12.60


In [21]:
# df 생성
prd_dict = {'품목':[], '가격':[], '세일가격':[]}
prd_df = pd.DataFrame(prd_dict)
prd_df

,품목,가격,세일가격


In [22]:
## 마지막 페이지 추출
# 요청 후 응답 객체의 코드를 bs4 객체로 생성

In [23]:
url

'http://jolse.com/category/toners-mists/1019?page=1'

In [24]:
bs_obj = get_request_product(url)

In [29]:
last = int(bs_obj.find('a', {'class':'last'})['href'].split('=')[1])
last

26

In [30]:
base_url = 'http://jolse.com/category/toners-mists/1019?page='
for i in range(1,last+1):
    url = base_url + str(i)
    # print(url)
    get_page_product(url)

http://jolse.com/category/toners-mists/1019?page=1
http://jolse.com/category/toners-mists/1019?page=2
http://jolse.com/category/toners-mists/1019?page=3
http://jolse.com/category/toners-mists/1019?page=4
http://jolse.com/category/toners-mists/1019?page=5
http://jolse.com/category/toners-mists/1019?page=6
http://jolse.com/category/toners-mists/1019?page=7
http://jolse.com/category/toners-mists/1019?page=8
http://jolse.com/category/toners-mists/1019?page=9
http://jolse.com/category/toners-mists/1019?page=10
http://jolse.com/category/toners-mists/1019?page=11
http://jolse.com/category/toners-mists/1019?page=12
http://jolse.com/category/toners-mists/1019?page=13
http://jolse.com/category/toners-mists/1019?page=14
http://jolse.com/category/toners-mists/1019?page=15
http://jolse.com/category/toners-mists/1019?page=16
http://jolse.com/category/toners-mists/1019?page=17
http://jolse.com/category/toners-mists/1019?page=18
http://jolse.com/category/toners-mists/1019?page=19
http://jolse.com/cate

- 수집할 때 수집되다가 특정 지점에서 에러 발생시 try ~ except 구분 활용해서 skip
    - 대량 데이터인 경우 한 두개 수집 에러는 너무 큰 문제로 생각 할 필요없다.